Importante 

Tener en cuenta bajadas de Procedimientos, Personas Armas, Divisas , vehiculos (secuestrados, ministerio), Narcotrafico general,Objetos y pegar en la carpeta bajadas de fecha del primero del 1 del mes en informar hasta fecha del informe 

### Importacion de librerias
importacion de libreias para tranformacion de datos

In [22]:
import pandas as pd
import re
import numpy as np
from openpyxl import load_workbook

### Parametros
Coloco los parametros a tener en cuenta para donde se encuentra los archivos de bajada

In [24]:
FECHA_INICIO = '01/10/2024'
FECHA_FINAL = '16/10/2024'

LUGARES_CATALOGADOS= {
                    "SECTOR RESTRINGIDO":	"SECTOR DE SEGURIDAD RESTRINGIDA AEROPORTUARIA",
                    "SECTOR PUBLICO"	:"INSTALACIONES DE ACCESO AL PUBLICO GENERAL",
                    "PERIMETRO DE JURISDICCION":	"PERIMETRO AEROPORTUARIO",
                    "FUERA DE JURISDICCION":	"DESPLIEGUE",
                    "TRANSPORTE PUBLICO": "TRANSPORTE PUBLICO",
                    "S/D": "S/D"
}

UNIDADES_MUNICIPIOS = {
    "EZE": "JOSÉ M. EZEIZA",
    "AER": "COMUNA 14",
    "SFO": "SAN FERNANDO",
    "BHI": "BAHÍA BLANCA",
    "MDP": "GENERAL PUEYRREDÓN",
    "MDZ": "LAS HERAS",
    "COR": "CAPITAL",
    "JUA": "9 DE JULIO",
    "LUI": "JUAN MARTÍN DE PUEYRREDÓN",
    "MLG": "MALARGÜE",
    "RAF": "SAN RAFAEL",
    "RCU": "RÍO CUARTO",
    "SRO": "CAPITAL",
    "JUJ": "EL CARMEN",
    "SAL": "LA CAPITAL",
    "CAT": "VALLE VIEJO",
    "LAR": "CAPITAL",
    "SGO": "CAPITAL",
    "TRH": "RÍO HONDO",
    "TUC": "CRUZ ALTA",
    "IGU": "IGUAZÚ",
    "FSA": "FORMOSA",
    "ROS": "ROSARIO",
    "RCQ": "GENERAL OBLIGADO",
    "POS": "CAPITAL",
    "CRR": "CAPITAL",
    "LIB": "PASO DE LOS LIBRES",
    "PAR": "PARANÁ",
    "RES": "SAN FERNANDO",
    "SVO": "LA CAPITAL",
    "BAR": "BARILOCHE",
    "TRE": "RAWSON",
    "NEU": "CONFLUENCIA",
    "CAL": "LAGO ARGENTINO",
    "CHP": "LÁCAR",
    "CRV": "ESCALANTE",
    "PMY": "BIEDMA",
    "ESQ": "FUTALEUFÚ",
    "GAL": "GÜER AIKE",
    "GDE": "RÍO GRANDE",
    "USU": "USHUAIA",
    "VIE": "ADOLFO ALSINA"
}

PROVINCIAS = {
    "C.A. BUENOS AIRES": "CIUDAD AUTONOMA DE BUENOS AIRES",
    "BUENOS AIRES": "BUENOS AIRES",
    "SANTA CRUZ": "SANTA CRUZ",
    "SANTA FE": "SANTA FE",
    "MENDOZA": "MENDOZA",
    "TUCUMÁN": "TUCUMAN",
    "TIERRA DEL FUEGO": "TIERRA DEL FUEGO ANTARTIDA E ISLAS DEL ATLANTICO SUR",
    "SALTA": "SALTA",
    "NEUQUÉN": "NEUQUEN",
    "CÓRDOBA": "CORDOBA",
    "LA PAMPA": "LA PAMPA",
    "RÍO NEGRO": "RIO NEGRO",
    "CATAMARCA": "CATAMARCA",
    "MISIONES": "MISIONES",
    "SAN JUAN": "SAN JUAN",
    "CORRIENTES": "CORRIENTES",
    "CHUBUT": "CHUBUT",
    "FORMOSA": "FORMOSA",
    "JUJUY": "JUJUY",
    "SANTIAGO DEL ESTERO": "SANTIAGO DEL ESTERO",
    "LA RIOJA": "LA RIOJA",
    "CHACO": "CHACO",
    "ENTRE RÍOS": "ENTRE RIOS",
    "SAN LUIS": "SAN LUIS"
}

PERSONAL = {    
    "DENUNCIA": 4,
    "CONTROL PREVENTIVO": 6,
    "PATIO VALIJAS": 6,
    "ALLANAMIENTO" :8,
}

SCANNER = {
    "DENUNCIA": 4,
    "CONTROL PREVENTIVO": 6,
    "PATIO VALIJAS": 6,
    "ALLANAMIENTO" :8,
}




### Funciones 
declaraciones de funciones procesar cada celda y columna del informe del sigipol como de operaciones

In [25]:
def procesar_causa_judicial(row):
    causa = row['CAUSAJUDICIALNUMERO']
    if pd.isna(causa) or causa in ["", "S/D", "A/S", "N/C"]:
        return f"{row['TIPO_CAUSA_INTERNA']}{row['CAUSA_INTERNA_NUMERO']}".strip()
    
    causa_str = str(causa).strip()
    
    prefijos = ["NRO.", "N°", "EXPT", "EXPEDIENTE", "CAUSA"]
    
    for prefijo in prefijos:
        if causa_str.upper().startswith(prefijo):
            causa_str = causa_str[len(prefijo):].strip()
    
    # Eliminar cualquier carácter que no sea número, letra o /ML
    causa_str = re.sub(r'[^\w/]', '', causa_str)
    
    return causa_str


contador_global_sigipol = {}
def generar_codigo_sigipol(row):
    id_operativo = row['ID_OPERATIVO']
    if id_operativo not in contador_global_sigipol:
        contador_global_sigipol[id_operativo] = 0
    contador_global_sigipol[id_operativo] += 1
    contador = contador_global_sigipol[id_operativo]
    return id_operativo + "(" + str(contador) + ")"

        
def procesar_descripcion(row):
    tipo = row['TIPO_PROCEDIMIENTO']
    if tipo == "DENUNCIA":
        return "DENUNCIA POLICIAL"
    elif tipo == "CONTROL PREVENTIVO":
        return f"CONTROL PREVENTIVO - {procesar_lugar(row)}"
    elif tipo == "ORDEN DE ALLANAMIENTO":
        return "ORDEN DE ALLANAMIENTO"
    elif tipo == "ORDEN DE ALLANAMIENTO / DETENCIÓN":
        return "ORDEN DE ALLANAMIENTO"
    else:
        return "OTRO MANDATO JUDICIAL"


def procesar_tipo(row):
    tipo = row['TIPO_PROCEDIMIENTO']
    if pd.isna(tipo):
        return ""
    elif tipo == "DENUNCIA"  or tipo == "CONTROL PREVENTIVO" :
        return "ORDEN POLICIAL"
    else:
        return "ORDEN JUDICIAL"
    
def procesar_provincia(row):
    provincia = row['PROVINCIA']
    if pd.isna(provincia):
        return ""
    return PROVINCIAS.get(provincia, provincia)
    
def procesar_municipio(row):
    unidad = row['UOSP']
    if pd.isna(unidad):
        return ""
    return UNIDADES_MUNICIPIOS.get(unidad, unidad)

def procesar_lugar(row):
    lugar = row['LUGAR_CATALOGADO_NIVEL_1']
    return  LUGARES_CATALOGADOS[lugar]

def procesar_direccion(row):
    lugar = row['LUGAR_CATALOGADO_NIVEL_1']
    if lugar == "FUERA DE JURISDICCION":
        return str(row['CALLE']) + " " + str(row['NUMERO']) + ", " + str(row['CIUDAD']) + " - " + str(row['PARTIDO'])
    else:
        return ""
    
def controlar_estado (row):
    ursa = row['URSA'] 
    unidad = row['UOSP'] 
    estado = row['ESTADO_PARTE'] 
    if pd.isna(unidad)  and ursa == 'RG4' and estado == 'NO DISPONIBLE ESTADISTICA':
        # print(f"Ursa: {ursa}, Unidad: {unidad}, Estado: {estado}")
        return  "DISPONIBLE ESTADISTICA"
    else:
        return estado 
    
def leer_excel_a_df(worksheet):
    data = []
    # Obtener los títulos de las columnas (fila 3, desde la columna B)
    titulos = [worksheet.cell(row=3, column=col).value for col in range(2, worksheet.max_column + 1)]
    # Leer los datos desde la fila 4 en adelante, empezando en la columna B
    for row in worksheet.iter_rows(min_row=4, min_col=2, max_col=worksheet.max_column, values_only=True):
        data.append(row)
    # Crear un DataFrame a partir de los datos
    return pd.DataFrame(data, columns=titulos)


# Definir una función para verificar si la fila de df_operaciones es diferente de alguna fila en df_base_informada
def filas_diferentes(row, df_base_informada):
    # Buscar si hay alguna fila en df_base_informada que sea idéntica en las columnas especificadas
    match = df_base_informada[
        (df_base_informada['ID_OPERATIVO'] == row['ID_OPERATIVO']) &
        (df_base_informada['FECHA'] == row['FECHA']) &
        (df_base_informada['HORA'] == row['HORA']) &
        (df_base_informada['LATITUD'] == row['LATITUD']) &
        (df_base_informada['LONGITUD'] == row['LONGITUD'])
    ]
    # Si no se encuentra un match, significa que la fila es diferente
    return match.empty



def limpiar_id_procedimiento(id_procedimiento):
    if isinstance(id_procedimiento, str):
        # Remover lo que está entre paréntesis y los paréntesis
        return re.sub(r'\(.*?\)', '', id_procedimiento).strip()
    return id_procedimiento

### Leer base de datos

leer la base de datos "data/Base_informada" 


In [26]:
df_base_informada = load_workbook("data/Base_informada.xlsx")
hoja = df_base_informada['GEOG. PROCEDIMIENTO']

# Crear un DataFrame a partir de los datos
df_base_informada = leer_excel_a_df(hoja)

### Ingresar bajadas

convierto las bajadas en el dataframe para poder manipularlos y poder hacer calculos

In [28]:
excel_bajada_procedimientos = pd.read_excel('bajadas/bajada_general.xls')
# excel_bajada_arma = pd.read_excel('bajadas/bajada_arma.xls')
# excel_bajada_vehiculos_secuestros = pd.read_excel('bajadas/bajada_vehiculos_secuestrados.xls')
# excel_bajada_vehiculos_secuestros_ministerio = pd.read_excel('bajadas/bajada_ministerio_vehiculos.xls')
# excel_bajada_personas = pd.read_excel('bajadas/bajada_general_persona.xls')
# excel_bajada_divisas = pd.read_excel('bajadas/bajada_divisas_secuestradas.xls')
# excel_bajada_narcotrafico = pd.read_excel('bajadas/bajada_general_narcotrafico.xls')

### Filtrar procedimientos 
solo que esta que estan disponible en estadistica y las fechas estipuladas de información 

In [29]:
fecha_inicio = pd.to_datetime(FECHA_INICIO, format='%d/%m/%Y')
fecha_final = pd.to_datetime(FECHA_FINAL, format='%d/%m/%Y')


excel_bajada_procedimientos['ESTADO_PARTE'] = excel_bajada_procedimientos.apply(controlar_estado ,axis=1).copy()
excel_bajada_procedimientos['UOSP'] = excel_bajada_procedimientos['UOSP'].fillna(excel_bajada_procedimientos['URSA'])
excel_bajada_procedimientos['GEOREFERENCIA_X'] = excel_bajada_procedimientos['GEOREFERENCIA_X'].fillna('-')
excel_bajada_procedimientos['GEOREFERENCIA_Y'] = excel_bajada_procedimientos['GEOREFERENCIA_Y'].fillna('-')

excel_bajada_procedimientos = excel_bajada_procedimientos[
    (excel_bajada_procedimientos['ESTADO_PARTE'] != 'NO DISPONIBLE ESTADISTICA') &
    (excel_bajada_procedimientos['DENUNCIAFECHA'] >= fecha_inicio) &
    (excel_bajada_procedimientos['DENUNCIAFECHA'] < fecha_final)
].copy()

### Filtros de procedimientos
Ejecucion de funciones para la bajada de procedimientos de sigipol  tranformando los datos para que se muestren como lo requiere la DNEC

In [30]:
df_procedimientos = pd.DataFrame()

df_procedimientos['ID_OPERATIVO'] = excel_bajada_procedimientos.apply(procesar_causa_judicial, axis=1)
df_procedimientos['ID_PROCEDIMIENTO'] = df_procedimientos.apply(generar_codigo_sigipol, axis=1)
df_procedimientos['UNIDAD_INTERVINIENTE'] = excel_bajada_procedimientos['UOSP']
df_procedimientos['DESCRIPCIÓN'] = excel_bajada_procedimientos.apply(procesar_descripcion, axis=1)
df_procedimientos['TIPO_INTERVENCION'] = excel_bajada_procedimientos.apply(procesar_tipo, axis=1)
df_procedimientos['PROVINCIA'] = excel_bajada_procedimientos.apply(procesar_provincia, axis=1)
df_procedimientos['DEPARTAMENTO O PARTIDO'] = excel_bajada_procedimientos.apply(procesar_municipio, axis=1)
df_procedimientos['LOCALIDAD'] = "-"
df_procedimientos['DIRECCION'] = excel_bajada_procedimientos.apply(procesar_direccion, axis=1)
df_procedimientos['LATITUD'] = excel_bajada_procedimientos['GEOREFERENCIA_Y']
df_procedimientos['LONGITUD'] = excel_bajada_procedimientos['GEOREFERENCIA_X']
df_procedimientos['FECHA'] = pd.to_datetime(excel_bajada_procedimientos['DENUNCIAFECHA'], errors='coerce').dt.strftime('%d/%m/%Y')
df_procedimientos['HORA'] = pd.to_datetime(excel_bajada_procedimientos['DENUNCIAFECHA'], errors='coerce').dt.strftime('%H:%M')
df_procedimientos['FUERZA_INTERVINIENTE'] = "PSA"
df_procedimientos['ZONA_SEGURIDAD_FRONTERAS'] = "-"
df_procedimientos['PASO_FRONTERIZO'] = "-"
df_procedimientos['OTRAS AGENCIAS INTERVINIENTES'] = "-"
df_procedimientos['Observaciones - Detalles'] = "-"


df_procedimientos = df_procedimientos[['FUERZA_INTERVINIENTE', 'ID_OPERATIVO', 'ID_PROCEDIMIENTO',
                                     'UNIDAD_INTERVINIENTE', 'DESCRIPCIÓN', 'TIPO_INTERVENCION',
                                     'PROVINCIA', 'DEPARTAMENTO O PARTIDO', 'LOCALIDAD', 'DIRECCION',
                                     'ZONA_SEGURIDAD_FRONTERAS', 'PASO_FRONTERIZO', 'LATITUD', 'LONGITUD',
                                     'FECHA', 'HORA', 'OTRAS AGENCIAS INTERVINIENTES', 'Observaciones - Detalles']]


df_procedimientos = df_procedimientos[
    ~df_procedimientos['ID_PROCEDIMIENTO'].isin(df_base_informada['ID_PROCEDIMIENTO'])
].copy()


### Filtros de Ordenes de servicios operaciones
Ejecucion de funciones para la planilla enviada por operaciones tranformando los datos para que se muestren como lo requiere la DNEC

In [31]:
excel_bajada_operaciones = load_workbook("bajadas/operaciones.xlsx")
hoja_geog = excel_bajada_operaciones['GEOG. PROCEDIMIENTO']
hoja_vehi = excel_bajada_operaciones['VEHI. Y PERSO. CONTROLADAS']
df_operaciones = leer_excel_a_df(hoja_geog)
df_operaciones = df_operaciones.dropna(subset=['ID_PROCEDIMIENTO'], how='any').copy()

df_operaciones['ID_PROCEDIMIENTO'] = df_operaciones['ID_PROCEDIMIENTO'].apply(limpiar_id_procedimiento)
df_operaciones['ID_OPERATIVO'] = df_operaciones['ID_PROCEDIMIENTO']



df_operaciones['FECHA'] = pd.to_datetime(df_operaciones['FECHA'], format='%Y-%m-%d', errors='coerce')
df_operaciones['HORA'] = pd.to_datetime(df_operaciones['HORA'], format='%H:%M', errors='coerce')



# # Convertir 'FECHA' y 'HORA' a string en ambos DataFrames
# df_operaciones['FECHA'] = df_operaciones['FECHA'].astype(str)
# df_base_informada['FECHA'] = df_base_informada['FECHA'].astype(str) # type: ignore

# df_operaciones['HORA'] = df_operaciones['HORA'].astype(str)
# df_base_informada['HORA'] = df_base_informada['HORA'].astype(str)# type: ignore


# Aplicar la función para identificar las filas que son diferentes
df_operaciones = df_operaciones[df_operaciones.apply(filas_diferentes, axis=1, df_base_informada=df_base_informada)].copy()


conteo_base_datos = df_base_informada['ID_OPERATIVO'].value_counts()
conteo_acumulado  = conteo_base_datos.to_dict()

for index, row in df_operaciones.iterrows():
    id_operativa = row['ID_OPERATIVO']
    
    # Verificar cuántas veces ha aparecido el ID_operativa en total hasta ahora (base + nuevos)
    if id_operativa in conteo_acumulado:
        conteo_acumulado[id_operativa] += 1
    else:
        conteo_acumulado[id_operativa] = 1
    
    # El nuevo ID_procedimiento será el ID_operativa más el número de veces que se ha repetido, dentro de paréntesis
    nuevo_id_procedimiento = f"{id_operativa}({conteo_acumulado[id_operativa]:02d})"
    
    # Asignar el nuevo ID_procedimiento solo en la columna ID_procedimiento
    df_operaciones.at[index, 'ID_PROCEDIMIENTO'] = nuevo_id_procedimiento


0   2024-04-04
1   2024-10-02
2   2024-10-02
3   2024-10-02
4   2024-10-02
Name: FECHA, dtype: datetime64[ns]
0               08:00:00
1    1900-01-03 08:02:24
2    1900-01-03 08:02:24
3    1900-01-03 17:02:24
4    1900-01-03 07:02:24
Name: HORA, dtype: object


### Filtros de procedimientos
Ejecucion de funciones para la bajada de procedimientos de sigipol  tranformando los datos para que se muestren como lo requiere la DNEC

In [9]:
df_procedimientos['ID_OPERATIVO'] = excel_bajada_procedimientos.apply(procesar_causa_judicial, axis=1)
df_procedimientos['ID_PROCEDIMIENTO'] = df_procedimientos.apply(generar_codigo_sigipol, axis=1)
df_procedimientos['UNIDAD_INTERVINIENTE'] = excel_bajada_procedimientos['UOSP']
df_procedimientos['DESCRIPCIÓN'] = excel_bajada_procedimientos.apply(procesar_descripcion, axis=1)
df_procedimientos['TIPO_INTERVENCION'] = excel_bajada_procedimientos.apply(procesar_tipo, axis=1)
df_procedimientos['PROVINCIA'] = excel_bajada_procedimientos.apply(procesar_provincia, axis=1)
df_procedimientos['DEPARTAMENTO O PARTIDO'] = excel_bajada_procedimientos.apply(procesar_municipio, axis=1)
df_procedimientos['LOCALIDAD'] = "-"
df_procedimientos['DIRECCION'] = excel_bajada_procedimientos.apply(procesar_direccion, axis=1)
df_procedimientos['LATITUD'] = excel_bajada_procedimientos['GEOREFERENCIA_Y']
df_procedimientos['LONGITUD'] = excel_bajada_procedimientos['GEOREFERENCIA_X']
df_procedimientos['FECHA'] = pd.to_datetime(excel_bajada_procedimientos['DENUNCIAFECHA'], errors='coerce').dt.strftime('%d/%m/%Y')
df_procedimientos['HORA'] = pd.to_datetime(excel_bajada_procedimientos['DENUNCIAFECHA'], errors='coerce').dt.strftime('%H:%M')
df_procedimientos['FUERZA_INTERVINIENTE'] = "PSA"
df_procedimientos['ZONA_SEGURIDAD_FRONTERAS'] = "-"
df_procedimientos['PASO_FRONTERIZO'] = "-"
df_procedimientos['OTRAS AGENCIAS INTERVINIENTES'] = "-"
df_procedimientos['Observaciones - Detalles'] = "-"


df_procedimientos = df_procedimientos[['FUERZA_INTERVINIENTE', 'ID_OPERATIVO', 'ID_PROCEDIMIENTO',
                                     'UNIDAD_INTERVINIENTE', 'DESCRIPCIÓN', 'TIPO_INTERVENCION',
                                     'PROVINCIA', 'DEPARTAMENTO O PARTIDO', 'LOCALIDAD', 'DIRECCION',
                                     'ZONA_SEGURIDAD_FRONTERAS', 'PASO_FRONTERIZO', 'LATITUD', 'LONGITUD',
                                     'FECHA', 'HORA', 'OTRAS AGENCIAS INTERVINIENTES', 'Observaciones - Detalles']]




### Guardado de informaciópn

Ordeno la informacion como de la hoja de procedimientos segun el la plainilla modelo y guardo en la planilla informe 

In [10]:
df_geog = pd.concat([df_operaciones, df_procedimientos])


nombre_archivo_modelo = "Planilla_modelo.xlsx"
nombre_archivo_informe = "Planilla_informe.xlsx"
wb = load_workbook(nombre_archivo_modelo)
ws = wb['GEOG. PROCEDIMIENTO']



# Escribir los datos del DataFrame desde la fila 4
for row_num, row in enumerate(df_geog.itertuples(index=False), start=4):
    for col_num, value in enumerate(row, start=2):  # Comienza desde la columna B (número 2)
        ws.cell(row=row_num, column=col_num).value = value

# Guardar los cambios
wb.save(nombre_archivo_informe)

print(f"\nArchivo Excel '{nombre_archivo_informe}' modificado con éxito.")


Archivo Excel 'Planilla_informe.xlsx' modificado con éxito.


Control de salidad de la informacion

In [11]:
# Aplicar la función procesar_causa_judicial a la copia

# # Contar cuántos casos se procesaron vs. cuántos se mantuvieron igual
# procesados = (wb_nueva_hoja['CAUSAJUDICIALNUMERO'] != wb_nueva_hoja['ID_OPERATIVO']).sum()
# print(f"\nCasos procesados: {procesados}")
# print(f"Casos sin cambios: {len(wb_nueva_hoja) - procesados}")


